### Imports

In [7]:
import tensorflow as tf
import numpy as np
# from keras.models import Sequential
from keras import layers, models, applications
# from IPython.display import display, Markdown, Latex
# from sklearn.datasets import make_blobs
# from matplotlib.widgets import Slider
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
tf.autograph.set_verbosity(0)

# Loading the dataset
The dataset is a collection of mfcc, mfcc derivative, chroma features, beat events, etc..


Feature extraction done using Librosa

In [12]:
train_data = np.load("D:/programming/python/speech recognition/competition/SpeachRecognition-AIC-/preprocessed_data/train_data.npz")
train_features = train_data['features']
train_labels = train_data['labels']

adapt_data = np.load("D:/programming/python/speech recognition/competition/SpeachRecognition-AIC-/preprocessed_data/train_data.npz")
adapt_features = adapt_data['features']
adapt_labels = adapt_data['labels']

print("Train data shape:", train_features.shape)
print("Adaptation data shape:", adapt_features.shape)

Train data shape: (50715, 166)
Adaptation data shape: (50715, 166)


### seeing the data

In [19]:
print(train_labels.shape)

(50715,)


# Building the model

## This model is designed as following:
### a combined vgg-16 model in series with transformer architecture

In [17]:


# Assume the input shape is (height, width, 1) for single-channel data
input_shape = (50715, 166, 1)  # Example shape; adjust according to your data

# Define the input layer with the new shape
inputs = tf.keras.Input(shape=input_shape)

# Convert single channel to 3 channels by repeating the input across the channel dimension
x = layers.Conv2D(3, (3, 3), padding='same')(inputs)

# Load the VGG16 model without the top layers or any weights
vgg_model = tf.keras.applications.VGG16(include_top=False, input_tensor=x, weights=None)
shapes_vgg = vgg_model.output

shapes_vgg
# we have to train the whole model so no need to freeze the weights 

<KerasTensor shape=(None, 1584, 5, 512), dtype=float32, sparse=False, name=keras_tensor_59>

### Transformer block:

In [16]:
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim)]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)


In [ ]:
num_heads = 8
embed_dim = shapes_vgg.shape[-1]  # Should match the last dimension of VGG output
ff_dim = 2048
transformer_layers = 4  # Number of transformer layers
num_classes = 10  # Number of output classes for speech recognition

# Reshape the VGG output to fit the transformer input requirements
x = layers.Reshape((-1, embed_dim))(x)

# Add Transformer layers
for _ in range(transformer_layers):
    x = TransformerBlock(embed_dim, num_heads, ff_dim)(x)

# Add Global Average Pooling and Output Layer
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(num_classes, activation='softmax')(x)

# Create the model
model = tf.keras.Model(inputs, outputs)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Display the model architecture
model.summary()
